# Google Play App's Review Data Scraper

> Aplikasi ini dibuat untuk melakukan penarikan data dari salah satu maupun beberapa aplikasi yang terdapat di Google Play Store ke dalam bentuk csv

Data yang diperoleh antara lain adalah :
- Review dari user
- Star Rating
- Hari dan Tanggal
- Informasi Aplikasi

## Setup

install packages yang dibutuhkan dan setup the imports:

(Digunakan untuk melakukan instalasi package yang diperlukan dalam melakukan data scraping)

In [24]:
%pip install -qq google-play-scraper
%pip install watermark

Note: you may need to restart the kernel to use updated packages.



In [25]:
%reload_ext watermark
%watermark -v -p pandas,matplotlib,seaborn,google_play_scraper

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.29.0

pandas             : 1.3.4
matplotlib         : 3.4.3
seaborn            : 0.11.2
google_play_scraper: 1.0.2



In [26]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

Aplikasi yang akan dilakukan data scraping, ditulis nama packagenya.
Dapat ditambahkan lebih dari 1 aplikasi

In [27]:
app_packages = [
  'id.co.bitcoin',
]

## Scraping App Information

Scrape informasi untuk setiap aplikasi:

In [28]:
app_infos = []

for ap in tqdm(app_packages):
  info = app(ap, lang='id', country='id')
  del info['comments']
  app_infos.append(info)

100%|██████████| 1/1 [00:00<00:00,  1.91it/s]


Merubah data hasil scraping ke dalam bentuk JSON

In [29]:
def print_json(json_object):
  json_str = json.dumps(
    json_object, 
    indent=2, 
    sort_keys=True, 
    default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

Data JSON dari aplikasi yang telah dilakukan scraping

In [30]:
print_json(app_infos[0])

{
  "adSupported": null,
  "androidVersion": "5.0",
  "androidVersionText": "5.0 dan yang lebih tinggi",
  "appId": "id.co.bitcoin",
  "containsAds": false,
  "contentRating": "Rating 3+",
  "contentRatingDescription": null,
  "currency": "IDR",
  "description": "The biggest Indonesia's Crypto Assets marketplace within your hands!\r\n\r\nIndodax Official Mobile App!\r\n\r\nIndodax is the biggest Crypto Asset marketplace in Indonesia. We currently have more than 2 million members from Indonesia and all over the world. With the sophisticated technology, Indodax has improved its mobile application which eases all traders with a wide range of crypto assets.\r\n\r\nWith this we proudly present\r\n\r\n\u2605Buy and Sell Crypto Assets\r\n\r\nIndodax gives you the opportunity to buy Bitcoin and other crypto-assets with less an hour. We have the most liquid transaction on all the markets with 2 pairs which are Indonesian Rupiah and USDT.\r\nTo support your trading activity, we also accept vario

Store Informasi aplikasi dengan converting JSON objects menjadi Pandas dataframe dan menyimpan hasilnya kedalam sebuah CSV file:

## Scraping App Reviews

In an ideal world, we would get all the reviews. But there are lots of them and we're scraping the data. That wouldn't be very polite. What should we do?

We want:

- Balanced dataset - roughly the same number of reviews for each score (1-5)
- A representative sample of the reviews for each app

We can satisfy the first requirement by using the scraping package option to filter the review score. For the second, we'll sort the reviews by their helpfulness, which are the reviews that Google Play thinks are most important. Just in case, we'll get a subset from the newest, too:

In [32]:
app_reviews = []

for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='id',
        country='id',
        sort=sort_order,
        count= 200 if score == 3 else 100,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

100%|██████████| 1/1 [00:04<00:00,  4.82s/it]


Note that we're adding the app id and sort order to each review. Here's an example for one:

In [33]:
print_json(app_reviews[0])

{
  "appId": "id.co.bitcoin",
  "at": "2021-12-19 18:43:41",
  "content": "Harga eminer kemarin di beberapa situs terkenal sampai 1000%, di indodax malah ga bergerak sama sekali, bikin jadi bertanya2, apakah ada permainan dari pihak indodax, bisa di bayangkan berapa uang yg harusnya di perileh oleh trader,,,perlu dipertanyakan lagi tentang kejujuran situs ini...",
  "repliedAt": "2021-12-19 19:07:59",
  "replyContent": "Dear member Indodax. Sisi kami tidak dapat mempercepat transaksi tersebut dikarenakan naik turunnya harga aset kripto mutlak ditentukan oleh permintaan dan penawaran yang ada. Anda dapat menunggu hingga harga memasuki antrian pada order book. Jika Anda mengalami kendala dalam bertransaksi, mohon langsung hubungi layanan support kami. Terima kasih.",
  "reviewCreatedVersion": "4.2.4",
  "reviewId": "gp:AOqpTOFUnbrc0l8pghNFMZEg373NiDzg8a1tLwNtntBtpQf_9jJh4DFWytBi8PWTpdmLhrD-ndkLeS9MwKKx_0k",
  "score": 1,
  "sortOrder": "most_relevant",
  "thumbsUpCount": 27,
  "userImage

`repliedAt` and `replyContent` contain the developer response to the review. Of course, they can be missing.

How many app reviews did we get?



In [34]:
len(app_reviews)

1200

Let's save the reviews to a CSV file:

In [19]:
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('../output/Hasil Scraping.csv', index=None, header=True)

## References

- [Google Play Scraper for Python](https://github.com/JoMingyu/google-play-scraper)